# Türkçe Spam SMS Veri Artırma (Dataset Generator)

Bu notebook, `Test/seed.csv` içindeki tohum verilerden Türkçe SMS sınıflandırma için sentetik (spam/ham) örnekler üretir ve `Test/augmented_sms.csv` dosyasına kaydeder.

- Parametreler (değiştirilebilir): `RANDOM_SEED`, `N_PER_SEED`, `DEDUP_THR`
- Çalışma klasörü: `Test/`
- Colab desteği için son hücrede yükleme/indirme yardımcıları vardır.



In [7]:
# 1) Ortam ve Yol Ayarları
from pathlib import Path
import csv, random, re

# Parametreler
RANDOM_SEED = 42
N_PER_SEED = 12
DEDUP_THR = 0.95

# Klasör/Yol (cwd içinde veya altında 'Test' ile uyumlu çalış)
BASE = Path.cwd()
if (BASE / "seed.csv").exists() or BASE.name.lower() == "test":
    pass
elif (BASE / "Test").exists():
    BASE = BASE / "Test"
else:
    BASE = BASE / "Test"

BASE.mkdir(parents=True, exist_ok=True)
SEED_FILE = BASE / "seed.csv"
OUT_FILE = BASE / "augmented_sms.csv"

# Rastgelelik sabitleme
random.seed(RANDOM_SEED)

print(f"Çalışma klasörü: {BASE}")
print(f"Tohum dosyası: {SEED_FILE}")
print(f"Çıktı dosyası: {OUT_FILE}")



Çalışma klasörü: c:\Users\emira\OneDrive\Belgeler\Cursor_git\BIL361\Test
Tohum dosyası: c:\Users\emira\OneDrive\Belgeler\Cursor_git\BIL361\Test\seed.csv
Çıktı dosyası: c:\Users\emira\OneDrive\Belgeler\Cursor_git\BIL361\Test\augmented_sms.csv


In [8]:
# 2) Yer Tutucular ve Şablonlar

BANKA = ["[BANKA]", "Bankanız", "Kart Sağlayıcınız"]
KARGO = ["[KARGO]", "Kargo Firması"]
OPERATOR = ["Operatörünüz", "Servis Sağlayıcı"]
PROMO = ["kampanya", "fırsat", "özel teklif"]
ACIL = ["hemen", "acilen", "şimdi", "vakit kaybetmeden"]
ACTION = ["tıklayın", "hemen giriş yapın", "başvurun", "onaylayın"]


def rand_amount():
    v = random.choice([49.9, 79, 129.9, 249, 499, 1299, 2499, 5000, 10000])
    style = random.choice(["{:.2f} TL","₺{:.2f}","{} TL","₺{}"])
    if "{}" in style:
        s = style.format(int(v))
    else:
        s = style.format(v).replace(".", random.choice([".", ","]))
    return s.replace("TL", random.choice(["TL"," tl"," TL"]))


def rand_code():
    return random.choice(["[KOD]","".join(random.choices("ABCDEFGHJKLMNPQRSTUVWXYZ2345679", k=6))])


def rand_link():
    return random.choice(["[LINK]","http://[LINK]","https://[LINK]"])


def rand_phone():
    return random.choice(["[TEL]","+90 5xx xxx xx xx","0 5xx xxx xx xx"])


def rand_date():
    return random.choice(["[TARIH]","31/10","01.11","02 Kas 10:30"])


def rand_time():
    return random.choice(["[SAAT]","10:30","14:00","18:45"])


SPAM_TEMPLATES = [
    "{banka} kartınıza {tutar} taksit {promo}! Detay için {acil} {link}",
    "Teslimat icin {kargo} {code} dogrulama gerekli. {acil} {action}: {link}",
    "Kredi onayiniz hazir! {tutar} limit. {acil} {action} {link}",
    "Hesabinizda riskli islem! {acil} {action} icin {code} kullanin: {link}",
    "{operator} {tutar} hediye internet! Aktivasyon icin {action}: {link}",
    "Cekilis kazandiniz! {tutar} hediye cekiniz var. {acil} talep edin: {link}",
]

HAM_TEMPLATES = [
    "[AD] aksam {saat} kütüphanede miyiz?",
    "Yarin {tarih} ders cikisi markete ugrar misin?",
    "Kargo {kargo} aradi, paket {code} ile {saat}'te gelecek.",
    "Toplanti {saat} gibi. Link: {link}",
    "Anne aradi, {saat} gibi geliyormus. Tel: {tel}",
    "[AD], bugun {saat} spor var mi?",
]



In [9]:
# 3) Gürültü, Doldurma ve Artırma Yardımcıları

def drop_turkish(text: str) -> str:
    return (
        text.replace("ş","s").replace("Ş","S")
            .replace("ı","i").replace("İ","I")
            .replace("ç","c").replace("Ç","C")
            .replace("ğ","g").replace("Ğ","G")
            .replace("ö","o").replace("Ö","O")
            .replace("ü","u").replace("Ü","U")
    )


def inject_noise(text: str) -> str:
    ops = []
    if random.random() < 0.5:
        ops.append(lambda s: s.lower())
    if random.random() < 0.5:
        ops.append(drop_turkish)
    if random.random() < 0.3:
        ops.append(lambda s: re.sub(r"\s{2,}", " ", re.sub(r"(\w)", r"\1 ", s, count=random.randint(1,3))))
    if random.random() < 0.4:
        ops.append(lambda s: s.replace("  ", " "))
    random.shuffle(ops)
    for op in ops[:random.randint(0, 2)]:
        text = op(text)
    return text


def fill_template(t: str, label: str) -> str:
    vals = {
        "banka": random.choice(BANKA),
        "kargo": random.choice(KARGO),
        "operator": random.choice(OPERATOR),
        "promo": random.choice(PROMO),
        "acil": random.choice(ACIL),
        "action": random.choice(ACTION),
        "tutar": rand_amount(),
        "code": rand_code(),
        "link": rand_link(),
        "tel": rand_phone(),
        "tarih": rand_date(),
        "saat": rand_time(),
    }
    out = t.format(**vals)
    return inject_noise(out)


def augment_row(text: str, label: str, n: int = N_PER_SEED):
    results = []
    for _ in range(n):
        t = random.choice(SPAM_TEMPLATES if label == "spam" else HAM_TEMPLATES)
        base = text if random.random() < 0.4 else None
        if base:
            msg = inject_noise(text)
        else:
            msg = fill_template(t, label)
        results.append(msg)
    return results


def normalize(s: str) -> str:
    s = s.lower()
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"[^\w\s]", "", s)
    return s.strip()


def jaccard(a: str, b: str, n: int = 3) -> float:
    A = {a[i:i+n] for i in range(len(a)-n+1)} if len(a) >= n else {a}
    B = {b[i:i+n] for i in range(len(b)-n+1)} if len(b) >= n else {b}
    return len(A & B) / max(1, len(A | B))


def dedup(texts, thr: float = DEDUP_THR):
    kept = []
    norms = []
    for t in texts:
        nt = normalize(t)
        if all(jaccard(nt, m) < thr for m in norms):
            kept.append(t)
            norms.append(nt)
    return kept



In [14]:
# 4) Çalıştırma: Seed oku, üret, deduplikasyon yap ve kaydet

# Seed oku
rows = []
with SEED_FILE.open(encoding="utf-8") as f:
    for r in csv.DictReader(f):
        rows.append({"text": r["text"].strip(), "label": r["label"].strip().lower()})

# Üretim
aug_texts, aug_labels = [], []
for r in rows:
    gen = augment_row(r["text"], r["label"], n=N_PER_SEED)
    aug_texts.extend(gen)
    aug_labels.extend([r["label"]]*len(gen))

# Tohum + sentetik
all_texts = [r["text"] for r in rows] + aug_texts
all_labels = [r["label"] for r in rows] + aug_labels

# Deduplikasyon
paired = list(zip(all_texts, all_labels))
random.shuffle(paired)
texts, labels = zip(*paired)
texts_dedup, labels_dedup = [], []
seen = []
for t, l in zip(texts, labels):
    nt = normalize(t)
    if all(jaccard(nt, s) < DEDUP_THR for s in seen):
        texts_dedup.append(t)
        labels_dedup.append(l)
        seen.append(nt)

# Kaydet
with OUT_FILE.open("w", encoding="utf-8", newline="") as f:
    w = csv.writer(f)
    w.writerow(["text","label"])
    for t,l in zip(texts_dedup, labels_dedup):
        w.writerow([t,l])

print(f"Yazdı: {OUT_FILE} (toplam {len(texts_dedup)} satır)")



Yazdı: c:\Users\emira\OneDrive\Belgeler\Cursor_git\BIL361\Test\augmented_sms.csv (toplam 101 satır)


In [ ]:
# 5) Colab Yardımcıları (isteğe bağlı)
try:
    from google.colab import files  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

print(f"Colab: {IN_COLAB}")

if IN_COLAB:
    print("seed.csv yüklemek için: files.upload() çağırın ve dosyayı seçin.")


def colab_upload_seed():
    if not IN_COLAB:
        print("Colab ortamında değilsiniz.")
        return
    uploaded = files.upload()
    for name in uploaded.keys():
        if name.lower().endswith(".csv"):
            import shutil
            shutil.move(name, str(SEED_FILE))
            print(f"Kaydedildi: {SEED_FILE}")


def colab_download_output():
    if not IN_COLAB:
        print("Colab ortamında değilsiniz.")
        return
    if OUT_FILE.exists():
        files.download(str(OUT_FILE))
    else:
        print("Önce veri üretin (4. hücreyi çalıştırın).")

